In [1]:
import numpy as np
import Feature_Engineering
import metrics
import train_test_split as tts
import matplotlib.pyplot as plt

In [2]:
file = './train.csv'
X_train, y_train = Feature_Engineering.datatransform(file)
X_train, y_train, X_test, y_test = tts.holdout(X_train, y_train)

In [3]:
Y_train = y_train.reshape((-1,1))
Y_test = y_test.reshape((-1,1))

In [4]:
print(X_train.shape, Y_test.shape)

(622, 14) (267, 1)


In [5]:
class layer():
    
    '''
    层。
    储存W,b,A,Z矩阵。
    提供forward（激活），backward（计算在输入值处的激活函数的导数）接口。
    -----
    params:
    Z : 输入值组成的矩阵。shape=(样本个数，本层节点个数)
    A : 激活值组成的矩阵。shape同Z
    B : 偏置单元
    dZ : 残差用于更新W
    dgZ : 激活函数在z点的导数值
    '''
    
    def __init__(self, method='test', nodenum=0, nextnodenum=0, size=0):
        
        self.method = method
        self.nodenum = nodenum
        self.nextnodenum = nextnodenum
        self.size = size
        self.W = np.ones((self.nodenum,self.nextnodenum))
        for i in range(self.nodenum):
            for j in range(self.nextnodenum):
                self.W[i][j] = np.random.random(1) ############### 直接用 np.random.random((self.nodenum, self.nextnodenum))
        #print(self.nextnodenum)
        self.b = np.ones((1,self.nextnodenum))
        for i in range(self.nextnodenum):
            self.b[0][i] = np.random.random(1)
        self.Z = np.ones((self.size,self.nodenum))
        for i in range(size):
            for j in range(self.nodenum):
                self.Z[i][j] = np.random.random(1)
        self.A = self.Z.copy()
        self.dZ = self.Z.copy()
        self.dgZ = self.Z.copy()
        self.B = np.ones((size,1))
        
    def _ReLU(self, Z, direction):
        if direction=='forward':
            for i in range(Z.shape[0]): ############### np.clip(Z, 0)
                for j in range(Z.shape[1]):
                    if Z[i][j]<=0:
                        Z[i][j] = 0
            return Z
        for i in range(Z.shape[0]):
                for j in range(Z.shape[1]):
                    if Z[i][j]<=0:
                        Z[i][j] = 0
                    else: 
                        Z[i][j] = 1
        return Z
    
    def _sigmoid(self, z, direction):
        haha = (1 / (1 + np.exp(-z)))
        if direction=='forward':
            return haha
        return haha*(1-haha)
    
    def _tanh(self, z, direction):
        if direction=='forward':
            return ((np.exp(2*z) - 1) / (np.exp(2*z) +1))
        return (4 * (np.exp(2*z) / ((np.exp(2*z) + 1)**2)))
    
    def _test(self, z, direction):
        if direction=='forward':
            return z
        return 1
    
        
    def forward(self):
        
        if self.method=='ReLU':
            self.A = self._ReLU(self.Z, direction='forward')
        elif self.method=='sigmoid':
            self.A = self._sigmoid(self.Z, direction='forward')
        elif self.method=='tanh':
            self.A = self._tanh(self.Z, direction='forward')
        elif self.method=='test':
            self.A = self._test(self.Z, direction='forward')
        return self
    
    
    def backward(self):
        
        if self.method=='ReLU':
            self.dgZ = self._ReLU(self.Z, direction='backward')
        elif self.method=='sigmoid':
            self.dgZ = self._sigmoid(self.Z, direction='backward')
        elif self.method=='tanh':
            self.dgZ = self._tanh(self.Z, direction='backward')
        elif self.method=='test':
            self.dgZ = self._test(self.Z, direction='backward')
        return self
    

In [6]:
class neuralnetwork():
    
    def __init__(self, X_train, Y_train, layer_num=3, eta=0.5, lamda=0.1):
        self.X_train = X_train
        self.Y_train = Y_train
        self.layer_num = layer_num
        self.eta = eta
        self.lamda = lamda
        #构建网络
        self.size = self.X_train.shape[0]
        self.layers = []
        #输入层：
        inlayer = layer(nodenum=X_train.shape[1], nextnodenum=20, size=self.size)
        self.layers.append(inlayer)
        self.__iter = 0
        #隐藏层：
        if self.layer_num==3:
            hidlayer = layer(nodenum=20, nextnodenum=self.Y_train.shape[1], size=self.size)
            ####################### 这里多了一个输出层，注释了就好了，你看看到底是哪里写错了吧（
            #outlayer = layer(nodenum=self.Y_train.shape[1], nextnodenum=self.Y_train.shape[1], size=self.size, method='sigmoid')#nextnodenum无意义
            self.layers.append(hidlayer)
            #self.layers.append(outlayer)
        else:
            for i in range(self.layer_num-3):
                hidlayer = layer(nodenum=20, nextnodenum=20, size=self.size)
                self.layers.append(hidlayer)
            hidlayer = layer(nodenum=20, nextnodenum=self.Y_train.shape[1], size=self.size)
        #输出层：
        outlayer = layer(nodenum=self.Y_train.shape[1], nextnodenum=2, size=self.size, method='sigmoid') #nextnodenum无意义
        self.layers.append(outlayer)
        #网络构建完毕。
    
    def forward(self):
        
        #对第一层：
        self.layers[0].A = self.X_train.copy()
        
        #接下来的每一层：
        for i in range(self.layer_num-1):
            self.layers[i+1].Z = self.layers[i].A.dot(self.layers[i].W) + self.layers[i].B.dot(self.layers[i].b)
            self.layers[i+1].forward()
        #print('3',self.layers[num-2-i].W)
            
        return self
    
    def backward(self):
        
        # 从输出层开始：
        self.layers[-1].backward()
        self.layers[-1].dZ = (self.layers[-1].A - self.Y_train) * (self.layers[-1].dgZ)
        
        # 依次往后传播,计算dz：
        num = self.layer_num #为了下面的代码简洁一点
        for i in range(num-1):
            self.layers[num-2-i].backward()
            self.layers[num-2-i].dZ = self.layers[num-1-i].dZ.dot(self.layers[num-2-i].W.T) * self.layers[num-2-i].dgZ
            
        # 更新W,b：
        
        for i in range(num-1):
            dW = (self.layers[num-2-i].W * self.lamda) + ((self.layers[num-2-i].A.T.dot(self.layers[num-1-i].dZ)) / self.size)
            db = np.sum((self.layers[num-1-i].dZ / self.size), axis=0).reshape(1,-1)
            if self.__iter%10==0:
                print('layer :',i,'\ndW:',dW[:3,:3],'\ndb:',db[:3],'\n',self.__iter)
            #print('1',self.layers[num-2-i].W)
            self.layers[num-2-i].W = self.layers[num-2-i].W - (self.eta * dW)
            #print('2',self.layers[num-2-i].b)
            self.layers[num-2-i].b = self.layers[num-2-i].b - (self.eta * db)
            
        self.__iter += 1
        
        return self

In [7]:
class NeuralNetworkClassifier():
    
    def __init__(self, network, X_train, Y_train, eta=0.5, lamda=0.1, epsilon=1e-4):
        self.network = network
        self.X_train = X_train
        self.Y_train = Y_train
        self.eta = eta
        self.epsilon = epsilon
        self.lamda = lamda
        self.mynetwork = neuralnetwork(self.X_train, self.Y_train)
        
    def _J(self, network, Y_train):
        y_pred = network.layers[-1].A
        W2 = 0
        for i in range(network.layer_num):
            W2 += np.sum(network.layers[i].W ** 2)
        J = 0.5 * np.sum((Y_train - y_pred)**2) + self.lamda * 0.5 * W2
        return J
        
    def fit(self):
        self.mynetwork.forward()
        _iter = 0
        #y_pred = mynetwork.layers[-1].A
        while(_iter<1000):
            J1 = self._J(Y_train=self.Y_train, network=self.mynetwork)
            self.mynetwork.backward()
            self.mynetwork.forward()
            J2 = self._J(Y_train=self.Y_train, network=self.mynetwork)
            #print('4',self.mynetwork.layers[0].W, mynetwork2.layers[0].W)
            print(J1)
            delta = J1 -J2
            
            if (delta) < self.epsilon: ########### 之前给delta加了abs，不应该加
                break
            
            _iter += 1
            if _iter%50==0:
                print('\ndelta=',delta)
        
        
    def predict(self, X_test, Y_test):
        self.newnetwork = neuralnetwork(X_train=X_test, Y_train=Y_test)
        for i in range(self.newnetwork.layer_num):
            self.newnetwork.layers[i].W = self.mynetwork.layers[i].W.copy()
            self.newnetwork.layers[i].b = self.mynetwork.layers[i].b.copy()
        self.newnetwork.forward()
        y_pred = self.newnetwork.layers[-1].A
        return np.array(y_pred>0.5, dtype='int')
    
    def score(self, X_test, Y_test, scoring=metrics.acc_score):
        y_pred = self.predict(X_test,Y_test).reshape(-1,).astype('float64')
        y_test = Y_test.reshape(-1,)
        print(y_pred, '1', y_test)
        return scoring(y_pred, y_test)

In [8]:
%%time
nnc = NeuralNetworkClassifier(network=neuralnetwork, X_train=X_train, Y_train=Y_train)
nnc.fit() # 最好也是能把loss的下降可视化出来

layer : 0 
dW: [[0.03451173]
 [0.04313441]
 [0.10007772]] 
db: [[0.00610534]] 
 0
layer : 1 
dW: [[0.08583717 0.01929017 0.07859025]
 [0.05617943 0.09532744 0.03585864]
 [0.03490155 0.03059819 0.05616403]] 
db: [[0.00181581 0.00249679 0.00607429 0.00583967 0.0058179  0.00563118
  0.00493673 0.0017669  0.00528832 0.00567335 0.00314871 0.00520634
  0.00344589 0.00119412 0.00086279 0.00412309 0.0008749  0.00351662
  0.00013963 0.0037975 ]] 
 0
117.49564112589744
117.26704213855587
117.06805062185948
116.83796504999168
116.53407284099569
116.13287184296397
115.62665627173604
115.01554089033583
114.29724865447939
113.45882342414296
layer : 0 
dW: [[0.02724945]
 [0.03136007]
 [0.05557984]] 
db: [[0.02112713]] 
 10
layer : 1 
dW: [[0.05137843 0.01165971 0.04768011]
 [0.03320194 0.05638497 0.01953647]
 [0.02064949 0.01780341 0.03188936]] 
db: [[ 3.15027085e-03  4.75752420e-03  1.27048949e-02  1.14849100e-02
   1.22325859e-02  1.15983463e-02  9.56085078e-03  3.51483399e-03
   1.14098467e-02  1.

In [9]:
y_pred = np.array(nnc.mynetwork.layers[-1].A > 0.5, dtype='int')
print(sum(y_pred==Y_train)/len(y_pred))

[0.74437299]


In [10]:
print(nnc.score(X_test, Y_test))

[0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0.] 1 [0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0.
 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0